# This notebook got no new idea from my side. A blunt code-along of whatever I learnt from StatQuest's decoder only tutorial in Pytorch.

In [13]:
import torch
import torch.nn as nn

from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader

In [2]:
#def embedding_gen(sentence):
sentence = "I need ya ya"
word_idx = {word:i for i, word in enumerate(sorted(set(sentence.split())))}
print(word_idx)

{'I': 0, 'need': 1, 'ya': 2}


In [3]:
token_to_id = {'what': 0,
                'is': 1,
                'statquest': 2,
                'awesome': 3,
                '<EOS>': 4}

id_to_token = dict(map(reversed, token_to_id.items()))

# Creating the dataset.

In [4]:
inputs = torch.tensor([[token_to_id["what"], # First input we wanna process
                        token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"]],
                        
                        [token_to_id["statquest"], # Second input we wanna process
                        token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"]]])

labels = torch.tensor([[token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]],
                        
                        [token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]]])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [5]:
def embedding_gen(word_idx):
    global embedding_dim, vocab_size
    embedding_dim = 16
    vocab_size = len(word_idx)
    embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
    global e_l
    e_l = []
    for i in range(vocab_size):
        e_l.append(i)
    e = torch.LongTensor(e_l)
    e = embeddings(e)
    return e

In [8]:
def positional_encodings(vocab_size, embedding_dim):
    pe = torch.zeros(vocab_size, embedding_dim)
    for pos in range(vocab_size):
        for i in range(0, 4, 2):
            em1 = torch.sin(torch.tensor(pos/(10000**((2*i)/embedding_dim))))
            em2 = torch.cos(torch.tensor(pos/(10000**((2*(i+1))/embedding_dim))))

            pe[pos][i] = em1
            pe[pos][i] = em2
    return pe

In [19]:
class positional_encodings(nn.Module):
    def __init__(self, d_model = embedding_dim, max_length = len(token_to_id)):
        super().__init__()

        pe = torch.zeros(max_length, d_model)
        for pos in range(max_length):
            for i in range(0, 4, 2):
                em1 = torch.sin(torch.tensor(pos/(10000**((2*i)/d_model))))
                em2 = torch.cos(torch.tensor(pos/(10000**((2*(i+1))/d_model))))

                pe[pos][i] = em1
                pe[pos][i] = em2

    def forward(self, word_embeddings):
        return word_embeddings + self.pe

In [14]:
softmax = nn.Softmax()

# Making the self-attention block; or reffered to as just the attention block as in the tutorial.
Extra point: We can later use this class to create multi-head attention blocks.

In [17]:
class Attention(nn.Module):
    def __init__(self, d_model):
        super().__init__()

        self.w_q = nn.Linear(in_features=d_model, out_features=d_model)
        self.w_k = nn.Linear(in_features=d_model, out_features=d_model)
        self.w_v = nn.Linear(in_features=d_model, out_features=d_model)

        self.row_dim = 0
        self.col_dim = 1

    def forward(self, q, k, v, mask=None):
        f_q = self.w_q(q)
        f_k = self.w_k(k)
        f_v = self.w_v(v)

        sims = torch.matmul(f_q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        scaled_sims = sims/torch.tensor(k.size(self.col_dim)**0.5)

        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)

        attention_percents = softmax(scaled_sims, dim=self.col_dim)
        attention_scores = torch.matmul(attention_percents, f_v)

        return attention_scores

In [21]:
 class DecoderOnlyTransformer(nn.Module):
    def __init__(self, num_tokens=4, d_model=embedding_dim, max_length=6):
        super().__init__()

        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)
        self.pe = positional_encodings(d_model)

        self.self_attention = Attention(d_model=d_model)
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):
        word_embeddings = self.we(token_ids)
        position_encoded = self.pe(word_embeddings)

        mask = torch.tril(torch.ones((torch_ids.size(dim=0)), (torch.size(dim=0))))
        mask = mask == 0

        self_attention_values = self.self_attention(position_encoded,
                                                    position_encoded,
                                                    position_encoded,
                                                    mask=mask)

        res_con_val = position_encoded + self_attention_values

        fc_layer_output = self.fc_layer(res_con_val)

        return fc_layer_output

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_idx):
        input_tokens, labels = batch
        output = self.forward(input_tokens[0])
        loss = self.loss(output, labels[0])
        return loss

In [ ]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=embedding_dim, max_len=6)

model_input = torch.tensor([[token_to_id["what"], # First input we wanna process
                        token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"]]]

input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor()

In [ ]:
torch.ceil(torch.tensor(221/2))

tensor(110.)